Coding the basic strategy

* Going to pass a lookup table 

* Will have to compare for similarities and differences in stipulation of basic strategy, and how differences in stipulation arise from minor variations in rules.

* For now, just do a really quick consistency scan against wikipedia

In [2]:
import pandas as pd
import random

columns = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'A']
index = [i for i in range(5, 22)]
index2 = ['A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']
index3 = ['2, 2','3, 3', '4, 4', '5, 5', '6, 6', '7, 7', '8, 8', '9, 9', '10, 10', 'A, A']
indices = index + index2 + index3

In [3]:
basic_strategy_table = pd.read_csv('basic-strategy.csv', header=None)
basic_strategy_table.index = indices
basic_strategy_table.columns = columns
basic_strategy_table

ValueError: Length mismatch: Expected axis has 39 elements, new values have 35 elements

In [329]:
def getcards(decks=1):
    """Brings out a number of sealed decks of cards in their original order to the virutal gaming table.

    Creates a dictionary tracking the name of each card, its quantity, and its value
    
    Keyword argument:
        decks {int} -- the number of 52-card decks that the casino is using (default 1)
    
    Returns:
        dictionary{keys: values} -- dictionary object where keys are the card name (e.g. 'K' for King )
                                    and where the values are a list object containing the card's quantity and its value.
                                    
    The value for the ace contains three elements, as aces can further take on the value of 1 or 11, depending on 
    the context of the hand. No distinction is made between card suits, i.e. spades, hearts, clubs, diamonds.
    """
    
    deck = {}
    total_cards = decks * 52
    card_quantity = int(total_cards / 13)
    
    deck['A'] = [card_quantity, 1, 11]

    for card in range(2, 11):
        deck[str(card)] = [card_quantity, card]
    
    for card in "JQK":
        deck[str(card)] = [card_quantity, 10]
        
    return deck, total_cards

deck, total_cards = getcards(decks=1)

In [330]:
def shuffler(deck, total_cards):
    
    """Shuffles the decks of cards, and places them inside the virtual gaming table card-shoe."""
    
    shuffled_shoe = []
    
    while len(shuffled_shoe) < total_cards:
        draw = random.choice(list(deck))
        if deck[draw][0] == 0:
            pass
        else:
            deck[draw][0] -= 1
            shuffled_shoe.append(draw)
            
    return shuffled_shoe

shuffled_shoe = shuffler(deck, total_cards)

In [331]:
rigged_shoe = []

for card in "2234A976636A3722":
    rigged_shoe.append(card)
print(rigged_shoe)

['2', '2', '3', '4', 'A', '9', '7', '6', '6', '3', '6', 'A', '9', '9', '6', '9']


In [332]:
def openinghand():
    
    """Deals two cards to the player, and one to the dealer from the shuffled shoe. Outputs a message if there is Blackjack
    
    Cards are taken sequentially from the shuffled card shoe, beginning with the last element of the shuffled shoe list object
    and working in reverse order."""
    
    player = []
    dealer = []

    player.append(rigged_shoe.pop())
    player.append(rigged_shoe.pop())
    
    dealer.append(rigged_shoe.pop())
    
    if ('A' in player) and (('10' in player) or ('J' in player) or ('Q' in player) or ('K' in player)):
        player_blackjack = 1
        print("Blackjack, house pays out at 3:2")
    else:
        player_blackjack = 0
        print("No Blackjack this time")
        
    return player, dealer, player_blackjack

player, dealer, player_blackjack = openinghand()

No Blackjack this time


In [333]:
def decode(action):
    if action == 'Sp':
        message = 'split'
    elif action == 'H':
        message = 'hit'
    elif action == 'D':
        message = 'double down'
    else:
        message = 'stick'
    return message

In [327]:
# O/S feature - need to define decision variable in the case of blackjack
# Queries remain and decisions need to be made in context of entire 
# system what value it should take, as leaving it blank yields a local
# variable referenced befroe assignment error.

# For now, set decision variable to 'S' i.e. stick and decide what to do with
# later on - ignoring it could yield some pathological behaviour later on.

# Okay great this is all working fine for a sample of cases that have been
# checked.

# Cosmetic - blackjack print message and hard/soft calculation in message

# HAVE TO CODE IN BUSTING

def basic_player(player, dealer, player_blackjack):
    
    if player_blackjack == 1:
        cardvalues = [21]
        decision = 'S'
        print("It's my lucky day")
    else:
        if player[0] == player[1]:
            lookup_pairs = player[0] + ", " + player[1]
            decision = basic_strategy_table.loc[lookup_pairs, dealer[0]]
            print("As I have a pair of {}s, I am going to {}".format(player[0], decode(decision)))
        elif ('A' in player):
            if player[0] == 'A':
                lookup_soft_total = player[0] + player[1]
            else:
                lookup_soft_total = player[1] + player[0]
                
            decision = basic_strategy_table.loc[lookup_soft_total, dealer[0]]
            print("As I have a soft total, I am going to {}".format(decode(decision)))
        else:
            cardvalues = []
            for card in player:
                cardvalues.append(deck[card][1])
                
            lookup_hard_total = sum(cardvalues)
            decision = basic_strategy_table.loc[lookup_hard_total, dealer[0]]
            print("I have a total of {}, I am going to {}".format(lookup_hard_total, decode(decision)))
    
    return decision

In [328]:
basic_player(player, dealer, player_blackjack)

KeyError: 'J'

In [270]:
# So the player either gets action = {blackjack(stick*), Hit, Double, Stick, Split}
# Start with the easiest cases and work up in order of difficulty, ie ones not involving recursion/feeding back

# S - stick, proceed to dealer's turn straightaway, completely deterministic
# D - double down - draw another card, and increase bet by 2x, then proceed to dealer's turn, which is completely deterministic
# H - hit - draw another card from the shuffled shoe and add it to player's hand, then have to perform another basic strategy lookup
# Sp - split - take hand and convert it into two hands, evaluate, requires another basic strategy lookup, and have to deal with
# mulitple hands being dealt, leading to multiple scores that need to be evaluated, and multiple payoffs and losses that also need to be calculated/

# Soft combos when there are more than two cards...

# A sequential decision tree has assisted here. With proper planning, 
# and once YOU are clear with the logic, the computer will be clear with
# the logic.

# Next module:

# Splits and multiple hands 
# Soft combos when there are more than 3 cards and repeated hitting - done, see separate notebook.

In [338]:
# Amended decision module with basic strategy that can handle additional
# cards being drawn, and also splits.

# O/S potential improvement - use a nested function rather than a more complicated function with quite intense logic? 

# Module for len(player) > 2 has been tried for a few cases for length 3

# Check P = ['A', '2', 'A'] and D = [9]
# Can't do A10 over three cards so have to add that permut. to pandas. - done, see separate notebook
# Have to also code in BUSTING - done, see separate notebook

# Just noticed that if a dealer has a picture card like 'Q', 'K', 'J', that won't be properly looked up in the table.
# So I will have to either amend the table to contain strings like K, Q, J; or I will have to do a value lookup first in the
# dictionary and use that to look up BS table. Good spot. Done, see separate notebook

def basic_player_multiple_hits(player, dealer, player_blackjack):
    
    if len(player) == 2:
        
        if player_blackjack == 1:
            cardvalues = [21]
            decision = 'S'
            print("It's my lucky day")
        else:
            if player[0] == player[1]:
                lookup_pairs = player[0] + ", " + player[1]
                decision = basic_strategy_table.loc[lookup_pairs, dealer[0]]
                print("As I have a pair of {}s, I am going to {}".format(player[0], decode(decision)))
            elif ('A' in player):
                if player[0] == 'A':
                    lookup_soft_total = player[0] + player[1]
                else:
                    lookup_soft_total = player[1] + player[0]
                
                decision = basic_strategy_table.loc[lookup_soft_total, dealer[0]]
                print("As I have a soft total, I am going to {}".format(decode(decision)))
            else:
                cardvalues = []
                for card in player:
                    cardvalues.append(deck[card][1])
                
                lookup_hard_total = sum(cardvalues)
                decision = basic_strategy_table.loc[lookup_hard_total, dealer[0]]
                print("I have a total of {}, I am going to {}".format(lookup_hard_total, decode(decision)))
        
    if len(player) > 2: 
        
        if 'A' in player:  # check if the >3 hand has an ace, i.e. is potentially soft.
            if player[0] != 'A':  # re-order dual ace to the left
                ace_index = player.index('A')  
                player[0], player[ace_index] = player[ace_index], player[0]
            
            cardvalues_no_ace = [deck[card][1] for card in player[1::]]
            
            if sum(cardvalues_no_ace) <= 10: # check soft totals
                lookup_soft_total = player[0] + str(sum(cardvalues_no_ace))

                decision = basic_strategy_table.loc[lookup_soft_total, dealer[0]]
            else:
                lookup_hard_total = deck['A'][1] + sum(cardvalues_no_ace) # variable naming, hard total as ace can only take value 1 now
                
                decision = basic_strategy_table.loc[lookup_hard_total, dealer[0]]
        else:
            cardvalues = [deck[card][1] for card in player]
            lookup_hard_total = sum(cardvalues)
            
            decision = basic_strategy_table.loc[lookup_hard_total, dealer[0]]
            
    return decision

In [340]:
# Action module
# player_action(decision)

# Issue is that in the for loop, player_action(decision) being called on each element of the decision list, containing 'H', 'H',
# ends up appending these cards to the original pre-split 

# Have amended to include player explicitly as an argument, rather than as a global variable lurking in the background,
# as it will need to be operated on. 

def player_action(player, decision):
    
    if decision == 'S':
        print(player)
        return player# -> go to dealerturn()

    if decision == 'D':
        drawcard = rigged_shoe.pop()
        player.append(drawcard)
        print(player) #-> go to dealerturn(), and also have to modify payout
        return player
        
    
    if decision == 'H':
        drawcard = rigged_shoe.pop()
        player.append(drawcard)
        print(player) #-> now there are three cards in player hand and one in dealer
        return player
        

    if decision == 'Sp': 
        multiple_hand_player = [player[i:i+1] for i in range(len(player))] #splits the player hand into two cards and places them in two sub-hands
    
        drawcard1 = rigged_shoe.pop()
        drawcard2 = rigged_shoe.pop()
    
        print("After, splitting, I receive {} and {} in each of my hands".format(drawcard1, drawcard2))
    
        multiple_hand_player[0].append(drawcard1)
        multiple_hand_player[1].append(drawcard2)
        
        print("My new hands are: " + str(multiple_hand_player))
        
        return multiple_hand_player

In [305]:
multiple_hand_player = player_action(player, decision)

After, splitting, I receive 3 and A in each of my hands
My new hands are: [['2', '3'], ['2', 'A']]


In [307]:
# Resolve the 1st split hand which has been dealt a card. 

decision = basic_player_multiple_hits(multiple_hand_player[0], dealer, player_blackjack)

I have a total of 5, I am going to hit


In [308]:
print(decision)

H


In [309]:
# So it takes [2, 3] and outputs the decision 'H', to hit.
# Pass BOTH [2, 3] AND the decision = 'H' back into the action module.
# It will take both inputs and operate on [2, 3] by adding a card to it.
# Then we reassign that output to the 0-th element of multiple_player_hand.

multiple_hand_player[0] = player_action(multiple_hand_player[0], decision)
print(multiple_hand_player)

['2', '3', '6']
[['2', '3', '6'], ['2', 'A']]


In [310]:
# Now the new [2, 3, 6] hand will need to be passed through the decision module again. 

decision = basic_player_multiple_hits(multiple_hand_player[0], dealer, player_blackjack)
print(decision)

D


In [311]:
multiple_hand_player[0] = player_action(multiple_hand_player[0], decision)
print(multiple_hand_player)

['2', '3', '6', '3']
[['2', '3', '6', '3'], ['2', 'A']]


In [312]:
decision = basic_player_multiple_hits(multiple_hand_player[0], dealer, player_blackjack)
print(decision)

H


In [313]:
multiple_hand_player[0] = player_action(multiple_hand_player[0], decision)
print(multiple_hand_player)

['2', '3', '6', '3', '6']
[['2', '3', '6', '3', '6'], ['2', 'A']]


In [314]:
decision = basic_player_multiple_hits(multiple_hand_player[0], dealer, player_blackjack)
print(decision)

S


In [315]:
multiple_hand_player[0] = player_action(multiple_hand_player[0], decision)
print(multiple_hand_player)

['2', '3', '6', '3', '6']
[['2', '3', '6', '3', '6'], ['2', 'A']]


In [ ]:
# So now the 1st hand has been resolved, and has concluded at [2, 3, 6, 3, 6], in keeping with the basic strategy. 

In [316]:
# Let's now resolve the 2nd hand which has been dealt a card. 

decision = basic_player_multiple_hits(multiple_hand_player[1], dealer, player_blackjack)

As I have a soft total, I am going to hit


In [317]:
print(decision)

H


In [318]:
# Above takes 2nd [2,A] hand and outputs decision = 'H' to hit.

# Pass BOTH [2, A] AND the decision = 'H' back into the action module.
# It will take both inputs and operate on [2, A] by adding a card to it.
# Then we reassign that output to the 1st element (subhand) of multiple_player_hand.

multiple_hand_player[1] = player_action(multiple_hand_player[1], decision)
print(multiple_hand_player)

['2', 'A', '6']
[['2', '3', '6', '3', '6'], ['2', 'A', '6']]


In [319]:
# Now the new [2, A, 6] hand will need to be passed through the decision module again. 

decision = basic_player_multiple_hits(multiple_hand_player[1], dealer, player_blackjack)
print(decision)

S


In [320]:
# Now the hand [2, A, 6] and decision = 'S' will need to be passed into the action module again.
multiple_hand_player[1] = player_action(multiple_hand_player[1], decision)

['A', '2', '6']


In [321]:
print(multiple_hand_player)

[['2', '3', '6', '3', '6'], ['A', '2', '6']]


In [ ]:
# Both hands have now been resolved, as the decision module has output 'S', which when passed to action module will move onto
# the dealer's turn.

# Automate the above process using loops to call the decision and action modules on each element of the list of hands.

for hand in multiple_hand_player:
    decision = basic_player_multiple_hits(multiple_hand_player[0], dealer, player_blackjack)
    multiple_hand_player[0] = player_action(multiple_hand_player[0], decision)

In [29]:
# Recursive piece that will be merged with basic_player_multiple_hits() under a separate section that deals with multiple hands
# Current strategy for coding this:
# 1. Create a list of lists, with each hand being a list, inside a list
# 2. Then have to evaluate each hand in this list, together with dealer card; using a lookup table.
# 3. Why not just call the basic_player_multiplehits() module on each list inside the list? Then each of the two hands will
# have a decision associated with it, which can be stored in a list with elements populated by decisions
# Flow and work with the logic as you go along via testing rather than pre-planning. Hacker/engineer style rather than academic

# A. player_blackjack variable is passed into basic_player decision modules.

# Dealing with the one split case where there are hits

In [205]:
# Iterates through multiple_hand list of list and calls the decision module on each of the subhands, stores respective decision
# in a decision list object. This will need to be passed through the action module.

# Note that the design choice here is repeatedly calling the decision module on each on the individual subhands, rather than
# including the multiple_hands list of lists as an additional argument of an extended decision module. But earlier plan
# was to merge this wit basic_player_multiple_hits(). No biggie, they may not be mutually incompatible objectives...we will see.

# Following the same principle, write a for loop that iterates through the list of lists and calls the action module on each
# subhand to carry out the actions. 

# For now looking at H only. S is trivial. D is also trivial. But then finally will have to deal with the possibility of multiple
# splits.

# This repeated calling of the decision module works fine, decision element is produced and added to decision list.
# There are no issues with the DM because it yields a consistent output, i.e. decision = {H, D, S, Sp} for each subhand.

for i in range(len(multiple_hand_player)):
    decision_list = []
    for i in range(2):
        decision_element = basic_player_multiple_hits(multiple_hand_player[i], dealer, player_blackjack)
        decision_list.append(decision_element)
print(decision_list)

I have a total of 5, I am going to hit
As I have a soft total, I am going to hit
I have a total of 5, I am going to hit
As I have a soft total, I am going to hit
['H', 'H']


In [165]:
# Take each output decision in the decision list and pass it through the player_action function so that cards are added to
# each hand

# However, this module is slightly different to the one above. Currently, calling the action module using each element of the
# decision list means that it operates on the initial player = [2,2] hand before splitting, so that cards are drawn from the
# shoe and placed in player, yielding the output [2, 2, 6, 3].

# Instead, we do not want the action module to add cards to the presplit [2, 2] hand, but to add the cards to each hand in
# multiple_hand_player. 

# Can either adjust current action module; or recode one that DOES meet my requirements

for i in range(len(decision_list)):
    
print(multiple_hand_player)

[['2', '3', ['2', '2', '6', '3']], ['2', 'A', ['2', '2', '6', '3']]]


In [176]:
multiple_hand_player

[['2', '3'], ['2', 'A']]

In [268]:
basic_player_multiple_hits(multiple_hand_player[0], dealer, player_blackjack)

I have a total of 5, I am going to hit


'H'

Notes

* The dimensionality of the problem is increasing a lot, feels a little combinatorial as splits create a branching factor. And there are complexities in the above aspects. I am investigating whether it is at all possible in a realistic timeframe to code this up solely using functions, in particuar because my attentions has been drawn to the fact that classes can keep track of states. 
* Follow up: With a little thought, and some pen/paper methods, the issue is that each split creates a branching factor of 2, i.e. it creates two separate player hands which individually need to be looked up, and following the lookup, there will be four possible actions for each hand, yielding a total of 8 possible actions. It can be easy to get lost in all of this. But there is a self-styled cybernetic principle, which is that when looking at the evolution of behaviour of a system, examine its constraints. The importance of this becomes clearer when we consider that this may be used to limit how far we need to expand combinatorially. 
* Strategy: The key constraints here, in order of the extent to which they limit the branching of the decision tree, are the number of positions on the table it is possible to open with a separate bet. And also player capital.
* Revision: On checking, it turns out there is no relationship between number of boxes/positions on the table, and the number of hands a player can split....
* Thought: Impose an artificial limit 
* Revision2: Came from the question of how frequently I will encounter multi-way splits. Coding practice suggests that one should not spend too much time coding for non-existent events. Abusing the boudnary between rare and non-existent, how frequently will multi-way splits occur? Was going to say look at the table, but examining it, there are some features of conditional probability/combinatorics I don't currently feel necessitates serious exploration. The key is next point.


* Strategy2: A constraint on the number of splits that will occur is the number of the "same" cards in the deck and the number of 52-card decks used. 
* Consider an arbitrary split on opening hand e.g. 2,2 for a 52-card deck. There will now be 2 hands, each containing one 2. If after hitting on both, either hand or both hands get a 2, then there will be 4 hands to play in total. This will be the maximum number of hands generated through splitting actions.
* Consider an arbitrary split on an opening hands again e.g. 2,2 for a 2 52-cards decks. Then there will now be a maximum of 8 possible hands being played. 
* Following same logic, for an arbitrary split, and using n 52-card decks, there will be a maxmimum of 4n hands being played, corresponding to 4 hands for every 52-card deck being used (as this defines the total number of the same card value in circulation).
* So for 6 decks, which is the standard for casino play, there will be a maxmimum of 24 possible hands resulting from splitting an arbitrary opening hand...


* Strategy3: Reduce unnecessary complexity, and impose artificial limit, as these cases may be extreme edge cases, in particular the maximum for 6 decks - 24 possible hands! However, only if it really is that hard to extend the look up table to more than 2 hands. 

Key detail I may have missed and need to check - the exact process when a split happens. Was under the impression that after splitting, you have two 1-card hands from which you can either stick, double, hit.

Think we assume that the the player is automatically given a card to turn 

Soft/hard hands:

* This required a few hours of thought...but I got there in the end and it is remarkably simple. Perhaps I would got it more quickly if I thought aloud. 
* Correct question - under what conditions does the ace have a dual value. 


With 2 card hands, state lookup and transition is completely specified by the basic strategy table.


For 3 card hands and higher:

Assume (or place it there for lookup purposes) that the ace is on the left.

If the sum of the remaining cards is less than or equal to 10, then this can be either 1 or 11, and is converted into A2 - A10, which can be looked up in the basic table.

If the sum of the remaing cards is greater than or equal to 11, then this becomes a hard total and is converted into 12 - 21, which can be looked up.

If there is more than one ace in the hand, only one ace will have the value 11. Use the above conditions to determine the lookup. 

Some extensions need to be appended to the BS lookup table. Will need to include A10 soft value. Has been amended.

EDGE CASES

Q: Can you hit more than once after you have split aces/receive more than one card? 

Q: Can you double after splitting aces? N0.

Q: Can you split non identical value 10 cards

Q: Can you split aces twice

Q: Why is A2A not 4? - for the consistency of my rule.

Q: Clarity on blackjack situations. In this case, all cards (in the event of a one player against dealer situation) are returned to the discard pile. 

Unless the dealer's card is ace or 10. In which case, some more complex rules apply. 

Will have to return to this as an amendment as don't want to disrupt flow

If the dealer's card is an ace, there is the possibility of a black-jack specific insurance side bet.  - all dealth with in ML journal.

Issue with 2 card decision module, to do with unbound variables. That's because I was passing a nonsensical input i.e. player = ['A', 'K'] AND player_blackjack = 0 simultaneously, so decision variable was not being bound as a result of this limbo condition.

But issue remains with multiple card hand decision module - solved. Typo when checking the number of cards in the hand.

Now have a roughshod basic_strategy decision module that can handle multiple cards in a hand, and can compute and lookup BS table with soft and hard hands. Well done! Now for more progress.